<a href="https://colab.research.google.com/github/Alinabelko/pt-labs/blob/master/pt_lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

   Каждая   разделяемая  библиотека  имеет  специальное  имя,  называемое
   "**soname**".  Оно  имеет  префикс "lib", само имя библиотеки, также слово
   ".so",  за  которым  следует  номер  стадии и номер версии библиотеки,
   которые  меняются  каждый  раз, когда у библиотеки меняется интерфейс.
   (Особое  исключение: библиотеки низкого уровня языка С не начинаются с
   "lib"). Полное имя soname включает в качестве префикса имя дериктории,
   в   которой   библиотека  находится.  На  живых  системах  полное  имя
   библиотеки  -  это  просто  символическая  ссылка  на  имя  real  name
   разделяемой библиотеки.

   Также,  каждая  разделяемая  библиотека имеет имя **real name**. Это - имя
   файла, содержащего собственно код библиотеки. Real name в дополнение к
   soname  содержит  номер  стадии, минорный номер (minor number), второй
   номер   стадии  и  номер  релиза.  Второй  номер  и  номер  релиза  не
   обязательны  (are  optional).  Минорный  номер и номер релиза дают вам
   знать,  какие  именно  версии  библиотек  установлены у вас в системе.
   Кстати,  в документации на библиотеку эти два последних номера могут с
   библиотечными своими собратьями не совпадать.

   Далее.   У   библиотеки   существует   ещё  один  тип  имени,  который
   используется  системой,  когда  библиотеку запрашивают. Называется оно
   **linker  name**,  которое  на  самом  деле  равно soname, но без указания
   версии.

   Способ   содержания   разделяемых  библиотек  в  хозяйстве  состоит  в
   различении   их   по   именам.  Каждое  приложение  должно  помнить  о
   разделяемой  библиотеке,  которая  ему нужна, помня только soname этой
   библиотеки.  Наоборот,  когда  вы  разделяемую библиотеку создаёте, вы
   имеете  дело  только  с  именем  файла,  содержащего  более  детальную
   информацию  о  версии.  Когда  вашу библиотеку будут устанавливать, её
   запишут  в  один  из  специально предназначенных для этого каталогов в
   файловой  системе  и запустят программу ldconfig. Эта хитрая программа
   смотрит  в  упомянутые специальные каталоги и создаёт soname-имена как
   символические  ссылки  к реальным именам, публикуя новости с фронтов в
   файле /etc/ld.so.cache.

libIsRising.so: linker name файл, симлинк на Fully qualified name или soname файл, в данном случае это libIsRising.so.0

libIsRising.so.0: soname, который указывает на файл с real name — libIsRising.so.0.0

libIsRising.so.0.0: непосредственно файл общей библиотеки

In [1]:
%%writefile libIsRising.cpp
#include <iostream>
using namespace std;

bool isRising(int arr[], int length)
{
      for(int i=0; i < length-1; i++)
    {
        if(arr[i] >= arr[i+1]){
            return false;
        };
    };
      return true;
}

Writing libIsRising.cpp


In [2]:
%%writefile libIsRising.h

bool isRising(int arr[], int length);


Writing libIsRising.h


In [0]:
%%script bash
#Создаём объектный файл, указав опцию PIC (Position Independent Code), 
#Warning (-Wall - warning all), -g для добавления дебаг-информации и -c — что бы создать только файл библиотеки, без вызова линкера:
g++ -fPIC -Wall -g -c libIsRising.cpp

In [4]:
%ls -l

total 52
-rw-r--r-- 1 root root   221 May  8 06:58 libIsRising.cpp
-rw-r--r-- 1 root root    38 May  8 06:59 libIsRising.h
-rw-r--r-- 1 root root 37136 May  8 06:59 libIsRising.o
drwxr-xr-x 1 root root  4096 May  4 16:26 sample_data/


In [0]:
%%script bash
# создадим общую библиотеку с дебагом (-g), тип общая (-shared),
# указав (с помощь -Wl) линковщику (/usr/bin/ld) на использование имени libhello и версии 0, 
# результат сохранить в файл libhello.so.0.0 (опция -o), и использовать объектный файл libhello.o:
gcc -g -shared -Wl,-soname,libIsRising.so.0 -o libIsRising.so.0.0 libIsRising.o -lc

In [6]:
%ls -l

total 80
-rw-r--r-- 1 root root   221 May  8 06:58 libIsRising.cpp
-rw-r--r-- 1 root root    38 May  8 06:59 libIsRising.h
-rw-r--r-- 1 root root 37136 May  8 06:59 libIsRising.o
-rwxr-xr-x 1 root root 28312 May  8 06:59 libIsRising.so.0.0*
drwxr-xr-x 1 root root  4096 May  4 16:26 sample_data/


In [7]:
%%script bash
#Далее — используем ldconfig, что бы создать файл с linker name, 
# который будет являться симлинком на файл с soname (полным именем), который в свою очередь — будет ссылкой на реальное имя:
ldconfig -v -n /content

/content:
	libIsRising.so.0 -> libIsRising.so.0.0 (changed)


In [8]:
%ls -l

total 80
-rw-r--r-- 1 root root   221 May  8 06:58 libIsRising.cpp
-rw-r--r-- 1 root root    38 May  8 06:59 libIsRising.h
-rw-r--r-- 1 root root 37136 May  8 06:59 libIsRising.o
lrwxrwxrwx 1 root root    18 May  8 06:59 libIsRising.so.0 -> libIsRising.so.0.0*
-rwxr-xr-x 1 root root 28312 May  8 06:59 libIsRising.so.0.0*
drwxr-xr-x 1 root root  4096 May  4 16:26 sample_data/


In [0]:
%%script bash
# Создаём символьную ссылку на файл (для linker name):
ln -sf libIsRising.so.0 libIsRising.so

In [10]:
%%writefile Main.cpp
#include <iostream>
#include <string>
#include "libIsRising.h"
using namespace std;

int main(){
    int length = 4;
    int arr[] = {1, 2, 3, 4};
    if(isRising(arr, length)){
        cout << "Входная последовательность строго возрастающая" << endl;
    } else {
        cout << "Входная последовательность не является строго возрастающей" << endl;
    }
    
  return 0;
}

Writing Main.cpp


In [0]:
%%script bash
#Собираем программу, указав через -l — имя библиотеки (без lib), и с помощью -L — путь, по которому необходимо искать файл библиотеки
g++ Main.cpp -o Main -lIsRising -L/content

In [12]:
%ls -l

total 100
-rw-r--r-- 1 root root   221 May  8 06:58 libIsRising.cpp
-rw-r--r-- 1 root root    38 May  8 06:59 libIsRising.h
-rw-r--r-- 1 root root 37136 May  8 06:59 libIsRising.o
lrwxrwxrwx 1 root root    16 May  8 06:59 libIsRising.so -> libIsRising.so.0*
lrwxrwxrwx 1 root root    18 May  8 06:59 libIsRising.so.0 -> libIsRising.so.0.0*
-rwxr-xr-x 1 root root 28312 May  8 06:59 libIsRising.so.0.0*
-rwxr-xr-x 1 root root 13112 May  8 06:59 Main*
-rw-r--r-- 1 root root   471 May  8 06:59 Main.cpp
drwxr-xr-x 1 root root  4096 May  4 16:26 sample_data/


In [0]:
%%script bash
#Добавляем текущий каталог в $LD_LIBRARY_PATH
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/content

In [14]:
%ls -l

total 100
-rw-r--r-- 1 root root   221 May  8 06:58 libIsRising.cpp
-rw-r--r-- 1 root root    38 May  8 06:59 libIsRising.h
-rw-r--r-- 1 root root 37136 May  8 06:59 libIsRising.o
lrwxrwxrwx 1 root root    16 May  8 06:59 libIsRising.so -> libIsRising.so.0*
lrwxrwxrwx 1 root root    18 May  8 06:59 libIsRising.so.0 -> libIsRising.so.0.0*
-rwxr-xr-x 1 root root 28312 May  8 06:59 libIsRising.so.0.0*
-rwxr-xr-x 1 root root 13112 May  8 06:59 Main*
-rw-r--r-- 1 root root   471 May  8 06:59 Main.cpp
drwxr-xr-x 1 root root  4096 May  4 16:26 sample_data/


In [15]:
%%script bash
#загрузчик динамических библиотек(динамический линковщик - dynamic linker) выдает ошибку
ldd Main

	linux-vdso.so.1 (0x00007ffd0da9e000)
	/usr/lib/x86_64-linux-gnu/libtcmalloc.so.4 (0x00007fef99f9c000)
	libIsRising.so.0 => not found
	libstdc++.so.6 => /usr/lib/x86_64-linux-gnu/libstdc++.so.6 (0x00007fef99c13000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007fef99822000)
	libunwind.so.8 => /usr/lib/x86_64-linux-gnu/libunwind.so.8 (0x00007fef99607000)
	libpthread.so.0 => /lib/x86_64-linux-gnu/libpthread.so.0 (0x00007fef993e8000)
	libm.so.6 => /lib/x86_64-linux-gnu/libm.so.6 (0x00007fef9904a000)
	/lib64/ld-linux-x86-64.so.2 (0x00007fef9a40f000)
	libgcc_s.so.1 => /lib/x86_64-linux-gnu/libgcc_s.so.1 (0x00007fef98e32000)
	liblzma.so.5 => /lib/x86_64-linux-gnu/liblzma.so.5 (0x00007fef98c0c000)
	libdl.so.2 => /lib/x86_64-linux-gnu/libdl.so.2 (0x00007fef98a08000)


In [16]:
%%script bash
#В этом списке описаны директории с библитеками. Кроме директории /lib - она подразумевается
cat /etc/ld.so.conf

include /etc/ld.so.conf.d/*.conf



In [0]:
%%script bash
#добавили текущую папку в конец файла
echo "/content" >> /etc/ld.so.conf

In [18]:
%%script bash
# система обновляет настройки
ldconfig

/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [19]:
print('Введите последовательность через пробел')
text = input()
arr = text.split()
print(arr)
f = open("Main.cpp","r+")
main = f.read();
main = main.replace('int length = 4;', 'int length = ' + str(len(arr)) + ';', 1)
text = text.replace(' ', ',')
main = main.replace('int arr[] = {1, 2, 3, 4};', 'int arr[] = {' + text +'};', 1)
f.close()
w = open("Main.cpp","w")
print(main)
w.write(main)
w.close()

Введите последовательность через пробел
5 4 23 1
['5', '4', '23', '1']
#include <iostream>
#include <string>
#include "libIsRising.h"
using namespace std;

int main(){
    int length = 4;
    int arr[] = {5,4,23,1};
    if(isRising(arr, length)){
        cout << "Входная последовательность строго возрастающая" << endl;
    } else {
        cout << "Входная последовательность не является строго возрастающей" << endl;
    }
    
  return 0;
}


In [0]:
%%script bash
#Собираем программу, указав через -l — имя библиотеки (без lib), и с помощью -L — путь, по которому необходимо искать файл библиотеки
g++ Main.cpp -o Main -lIsRising -L/content

In [21]:
%%script bash
./Main

Входная последовательность не является строго возрастающей


In [22]:
%%script bash
cat Main.cpp

#include <iostream>
#include <string>
#include "libIsRising.h"
using namespace std;

int main(){
    int length = 4;
    int arr[] = {5,4,23,1};
    if(isRising(arr, length)){
        cout << "Входная последовательность строго возрастающая" << endl;
    } else {
        cout << "Входная последовательность не является строго возрастающей" << endl;
    }
    
  return 0;
}